# Домашнее задание I

В этом домашнем задании вы будете генерировать синтетические точечные процессы и анализировать реальные наборы данных. 

Максимальное количетсво баллов - 25, проходной балл - 10.

На что можно опереться:
- Семинары
- https://www.math.fsu.edu/~ychen/research/Thinning%20algorithm.pdf

## Задание 1 (15 баллов)

- Создайте набор данных гомогенного процесса Пуассона с $\lambda_c = 10/(c+1)$, где $c\in\{0,1,2,3,4\}$ — тип события. Максимальное время $T = 10$. Вы можете сделать это вручную или с помощью библиотеки тиков.

- Создайте набор данных неоднородного процесса Пуассона с $\lambda_c(t) = 10 e^{-\frac{0.2}{c+1}t} (1+sin(5t))/(c+1)$ , где $c \in\{0,1,2,3,4\}$ — тип события. Максимальное время $T = 10$. Вы можете сделать это вручную или с помощью библиотеки тиков.

- Сгенерируйте процесс Хоукса с 5 типами событий $c\in\{0,1,2,3,4\}$ и максимальным временем $T=10$:
   - Базовая интенсивность: $\mu_{c} = \frac{5}{c+1}$
   - Ядра:
    - $\phi_{0,j} = 0.1^j((j + 0.1) * 10 + t)^{-2^{1/(j+1)}}$
    - $i \in \{1,2,3\}$: $\phi_{i,j} = \sum_{d = 1}^{3} 0.1^{i+j+d} 2^{d/(j+1)}\exp(-2^{d/(j+1)} t)$
    - $j\in\{0,1,2,3\}$: $\phi_{4,j} = 0.5^{1/(j+1)}*(1+sin(5^{1/(j+1)}t))$
    - $\phi_{4,4} = 0$

Создайте 2000 последовательностей в каждом наборе данных. Используйте следующие переменные: ```h_poisson_dset```, ```i_poisson_dset```, ```hawkes_dset```.

### Ваше решение 

Похожие задачи были решены в семинаре 1. Обратите внимание, что сгенерить последовательность с несколькими типами событий эквивалентно тому, чтобы сгенерить событие с интенсивность $\lambda = \sum_c \lambda_c$ и выбрать его тип с вероятностями $\lambda_c/\lambda$.

## Загрузка реальных данных

### Набор данных ретвитов

Набор данных с историей ретвитов 3 категорий пользователей как тип события.

In [3]:
!pip install googledrivedownloader

You should consider upgrading via the 'C:\Users\Vladislav\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1q1VUY3VTmaZqT8VjWY5hKrSx8v9tZbt7',
                                    dest_path='./train_tick.pkl')

In [5]:
!pip install pickle-mixin --upgrade

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6034 sha256=b911a2f3224d41769c74b69b05ff6f685835cabc558bb5ad41a720526c57c961
  Stored in directory: c:\users\vladislav\appdata\local\pip\cache\wheels\58\c6\8b\061bd4edc8cea2b2235758c3e50473d08499236fbfdd21e6b3
Successfully built pickle-mixin


You should consider upgrading via the 'C:\Users\Vladislav\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
import pickle

with open('./train_tick.pkl', 'rb') as f:
  tweets_dset = pickle.load(f)

Количество типов событий:

In [7]:
print('N types =', len(tweets_dset[0]))

N types = 3


Размер набора данных:

In [ ]:
len(tweets_dset)

20000

### Amazon датасет

Набор данных Amazon с историей покупок. Типы событий — это категории продуктов.

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1nlp2wQAPrNHst7to0vkkz9uHyI4d1-xU',
                                    dest_path='./amz7k.zip')

In [ ]:
!unzip amz7k.zip -d .

In [ ]:
from typing import List
import os
import tqdm
import torch
import pandas as pd

def load_data(
    path_to_data: str,
    n_types: int,
    return_times = False
    ) -> List[torch.Tensor]:
    files = os.listdir(path_to_data)
    seqs = []
    if 'clusters.csv' in files:
        files.remove('clusters.csv')
    for i, f in tqdm.tqdm(enumerate(files)):
        df = pd.read_csv(path_to_data + '/' + f)
        df = df.sort_values(by=['time'])
        seq = torch.zeros(len(df['time']), n_types + 1)
        for id_, (time, event) in enumerate(zip(list(df['time']), list(df['event']))):
            seq[id_,0] = time
            seq[id_, int(event) + 1] = 1
        if return_times:
            seq[1:, 0] -= seq[:-1, 0]
            seq[0, 0] = 0
        else:
            seq[:,0]-=seq[0,0].clone()
        seq[:,0]/=86400 # the input is in unix time
        seqs.append(seq)
    return seqs

In [ ]:
data = load_data('amz7k', 8)

7523it [00:41, 180.99it/s]


In [ ]:
data[0].shape

torch.Size([192, 9])

In [ ]:
n_types = 8
N = len(data)
print(N)

7523


In [ ]:
def torch_tensor_to_tick(data, n_types):
  dset = []
  for seq in data:
    tick_seq = []
    for i in range(n_types):
      tick_seq.append(seq[seq[:,i+1] == 1, 0].numpy())
    dset.append(tick_seq)
  return dset


In [ ]:
amazon_dset = torch_tensor_to_tick(data, n_types)

In [ ]:
amazon_dset[0]

[array([ 816., 1439., 1440., 1661., 1749.], dtype=float32),
 array([   0.,   63.,   68.,  115.,  188.,  336.,  336.,  352.,  369.,
         476.,  478.,  831.,  832.,  832.,  863.,  863.,  873.,  873.,
         873.,  887.,  960.,  960.,  961.,  969.,  969., 1112., 1119.,
        1134., 1136., 1265., 1279., 1285., 1285., 1304., 1305., 1305.],
       dtype=float32),
 array([], dtype=float32),
 array([1095., 1121., 1122., 1168., 1203., 1326., 1551., 1572., 2101.,
        2223., 2382.], dtype=float32),
 array([ 104.,  211.,  270.,  365.,  381.,  456.,  547.,  566.,  666.,
         740.,  767.,  792.,  845.,  850.,  909.,  981.,  995.,  995.,
         995.,  995., 1036., 1068., 1123., 1124., 1246., 1428., 1442.,
        1468.], dtype=float32),
 array([], dtype=float32),
 array([  30.,  497.,  664.,  677.,  686.,  691.,  744.,  750.,  755.,
         757.,  757.,  768.,  768.,  769.,  774.,  775.,  775.,  783.,
         789.,  789.,  796.,  799.,  800.,  803.,  803.,  821.,  828.,
         9

## Вопрос

Какого поведения вы ожидаете от ретвитов и наборов данных Amazon?

### Ваш ответ

## Заданиие 2 (10 баллов)

Для каждого набора данных постройте график:
- Функции подсчета для 50 последовательностей на одном графике, как вы можете объяснить различное поведение? .
- Эмпирическую PDF для времени возврата без учета типа события и PDF однородного Пуассона в предположении, что этот набор данных является пуассоновским. Они близки? Какое поведение имеет эмпирическая PDF?
- Эмпирическую функция выживания, игнорирующую тип события, и функцию выживания однородного пуассона, предполагающую, что этот набор данных является пуассоновским.
- Графики распределения типов событий.

_Примечание: реальные данные могут иметь выбросы, которые могут снизить качество ваших графиков. Вы должны отфильтровать их, если они у вас есть._

### Ваше решение